<a href="https://colab.research.google.com/github/Kallystos/Alura-Google/blob/main/SADOC_Sistema_de_Aux%C3%ADlio_para_Descobrimento_de_Obras_Culturais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [206]:
# Instalar Framework ADK de agentes do Google ################################################
%pip -q install google-adk

In [207]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [208]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [209]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [210]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico indicado.
        Busque notícias no tópico inserido nas áreas de filmes, séries, documentários, livros, música e jogos eletrônicos.
        Foque em no máximo 5 notícias relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        """,
        description="Agente que busca notícias sobre o tópico indicado no Google",
        tools=[google_search]
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [211]:
######################################
# --- Agente 2: Redator do Post --- #
######################################
def agente_redator(topico, lancamentos_buscados):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo que responde ajudando seus usuários a encontrarem novas músicas, filmes, etc.
            Utilize os lançamentos buscados mais relevantes e, com base nisso, dê até no máximo 10 sugestões de obras sobre o tema indicado.
            As sugestões devem conter no mínimo:
            Uma de filme, uma de série, uma de documentário, uma de livro, uma de música e uma de games (jogo eletrônico).
            A primeira sugestão deve ser a "sugestão destaque", ou seja, a mais relevante.
            As sugestões devem ser as mais recentes possíveis, de lançamentos.
            Caso não haja lançamentos, pode ser o resultado mais relevante.
            """,
        description="Agente redator de resposta"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nSugestões: {lancamentos_buscados}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [212]:
##########################################
# --- Agente 3: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Responda apenas a resposta final diretamente para o usuário sobre o tema abordado.
            """,
        description="Agente revisor de resposta final para usuário."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [213]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Auxílio para Descobrimento de Obras Culturais com 3 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você deseja receber novidades e dicas culturais: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Então vamos sugerir algumas coisinhas pra você sobre {topico}!")

    texto_revisado = agente_revisor(topico, rascunho)
    print("\n--- Resultado do Agente 3 (Revisor): ---\n")
    display(to_markdown(texto_revisado))
    print("------------------------------------")

🚀 Iniciando o Sistema de Auxílio para Descobrimento de Obras Culturais com 3 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você deseja receber novidades e dicas culturais: aviação
Maravilha! Então vamos sugerir algumas coisinhas pra você sobre aviação!

--- Resultado do Agente 3 (Revisor): ---



> E aí, pessoal! Curiosos sobre o mundo da aviação? Preparei uma lista de filmes, séries, livros e outras mídias pra vocês ficarem por dentro das novidades e lançamentos do setor. Confiram só:
> 
> **Destaque:**
> 
> *   **Filme:** "Top Gun 3" (em produção) - Preparem-se para decolar! A sequência de "Top Gun" está vindo aí com novas tecnologias e cenas de voo de tirar o fôlego.
> 
> **Outras Sugestões:**
> 
> *   **Série:** "Mayday! Desastres Aéreos" (Novos Episódios) - Pra quem curte entender os bastidores dos acidentes aéreos e a segurança da aviação.
> 
> *   **Documentário:** "Space X: Retorno ao espaço" (Netflix, 2022) - Uma missão inspiradora da Space X com a NASA que você não pode perder.
> 
> *   **Livro:** "Aeronaves Embraer: Uma História de Sucesso" de Carlos Henrique Cristilli - Pra conhecer a trajetória da fabricante brasileira Embraer.
> 
> *   **Música:** "Learn to Fly" - Foo Fighters - O clipe dessa música é uma viagem no mundo da aviação!
> 
> *   **Game:** "Microsoft Flight Simulator" (2020) - Pra quem quer pilotar e explorar paisagens incríveis sem sair de casa.
> 
> *   **Evento:** LABACE (2025) - Fiquem ligados no maior evento de aviação executiva da América Latina.
> 
> *   **Podcast:** "Asas e Hélices" - Notícias, histórias e entrevistas com especialistas do mundo da aviação.
> 
> *   **Canal do YouTube:** "Aviation Analysis" - Análises técnicas de acidentes aéreos e segurança de voo pra você ficar por dentro.
> 
> *   **Museu:** Museu Aeroespacial (Rio de Janeiro) - Uma coleção de aeronaves e artefatos históricos pra quem ama a história da aviação.


------------------------------------
